# Section 1-2 - Creating Dummy Variables

이전 섹션들에서, 우리는 categorical value들인 Embarked column의  {C, S, Q}를 numerical value인 {1, 2, 3}으로 바꿨다. 그러자 후자의 numerical value는 ordering 문제가 생긴다(전자와는 다르게). 이 문제를 해결하기 위해, dummy variable 컨셉을 도입해야 한다.

# Pandas - Extracting data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/train.csv')

# Pandas - Cleaning data

In [2]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

from scipy.stats import mode

mode_embarked = mode(df['Embarked'].tolist())[0][0]
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

C:\Users\edte1\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Sex column에는 오직 두 가지의 유일한 값들만 있기 때문에 ordering 문제가 없다.

In [3]:
df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

그러나 Embarked column의 경우에는 {C, S, Q}를 {1, 2, 3}으로 변환했기 때문에 C < S < Q의 순서가 되는 ordering 문제가 나타나게 된다. 원래는 알파벳 순서대로 배열했던 것 뿐이기 때문에 ordering 문제를 해결해야 한다.

이 문제를 해결하기 위해서는, dummy variable을 도입해야 한다. 새로운 column들을 필수적으로 만들어야 하는데, C에 대해서 True/False, S에 대해서 True/False, Q에 대해서 True/False를 각각 1/0으로 표현해야 하기 때문이다. Pandas에는 이러한 column들을 자동적으로 만들어주는 내부 함수가 있다.

In [4]:
pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0
6,0,0,1
7,0,0,1
8,0,0,1
9,1,0,0


이렇게 생성한 dummy variable의 column들을 메인 DataFrame와 concatenate한다.

In [8]:
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

#### Exercise

* Sex column의 두 개 값에 대해 dummy variable을 생성해보자

In [9]:
df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]

전처리 된 training data를 살펴보자

In [10]:
df.head(10)

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Embarked_C,Embarked_Q,Embarked_S
0,0,1,3,22.000000,1,0,7.2500,1,0,0,1
1,1,2,1,38.000000,1,0,71.2833,0,1,0,0
2,1,3,3,26.000000,0,0,7.9250,0,0,0,1
3,1,4,1,35.000000,1,0,53.1000,0,0,0,1
4,0,5,3,35.000000,0,0,8.0500,1,0,0,1
5,0,6,3,29.699118,0,0,8.4583,1,0,1,0
6,0,7,1,54.000000,0,0,51.8625,1,0,0,1
7,0,8,3,2.000000,3,1,21.0750,1,0,0,1
8,1,9,3,27.000000,0,2,11.1333,0,0,0,1
9,1,10,2,14.000000,1,0,30.0708,0,1,0,0


In [11]:
train_data = df.values

# Scikit-learn - Training the model

In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100)
model = model.fit(train_data[0:, 2:], train_data[0:, 0])

# Scikit-learn - Making predictions

In [14]:
df_test = pd.read_csv('../data/test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means.loc[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

test data에 대해서도 dummy variable을 생성한다.

In [16]:
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')], axis=1)

In [18]:
df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:, 1:])

# Pandas - Preparing for submission

In [19]:
result = np.c_[test_data[:, 0].astype(int), output.astype(int)]

df_result = pd.DataFrame(result[:, 0:2], columns=['PassengerID', 'Survived'])
df_result.to_csv('../results/titanic_test_1-2.csv', index=False)